In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas import read_csv
from numpy import set_printoptions

In [ ]:
os.chdir(r'CMSC-197-Miniproject')

In [ ]:
data = pd.read_csv("amazon_reviews(raw).txt", delimiter = "\t")

In [14]:
#data.loc[data["LABEL"] == "__label1__", "LABEL"] = 0
#data.loc[data["LABEL"] == "__label2__", "LABEL"] = 1

In [15]:
data.head()

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,__label1__,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,__label1__,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [ ]:
# Preparing Data
# Getting only relevant dimensions (Label, Rating, Verified Purchase, Review title and Review text) necessary for data analysis
amazon_reviews = pd.read_csv("amazon_reviews.txt(raw)", delimiter = "\t")
df = amazon_reviews[['LABEL','REVIEW_TITLE', 'REVIEW_TEXT']]
df.rename(columns = {'LABEL': 'Label', 'REVIEW_TITLE': 'Review_Title', 'REVIEW_TEXT': 'Review_Text'}, inplace = True)

C:\Users\paolo\AppData\Local\Temp\ipykernel_13496\4061611525.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'LABEL': 'Label', 'REVIEW_TITLE': 'Review_Title', 'REVIEW_TEXT': 'Review_Text'}, inplace = True)


In [17]:
df['Review_Text'] = df['Review_Title'] + " " + df['Review_Text']

C:\Users\paolo\AppData\Local\Temp\ipykernel_13496\433396366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Review_Text'] = df['Review_Title'] + " " + df['Review_Text']


In [18]:
df=df.drop(['Review_Title'], axis=1)

In [19]:
# Remove N/A
for col in df.columns:
    print(col, df[col].isnull().sum())
    df = df.dropna()

Label 0
Review_Text 0


In [20]:
%pip install contractions
import contractions

   ---------------------------------------- 0.0/289.9 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/289.9 kB 1.3 MB/s eta 0:00:01
   ------------------------------- -------- 225.3/289.9 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 289.9/289.9 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
# Expansion literary shortcuts -> e.g., I've to I have
df['no_contract'] = df['Review_Text'].apply(lambda x: [contractions.fix(word) for word in x.split()]) 
df['REVIEW_TEXT_str'] = [' '.join(map(str, l)) for l in df['no_contract']]

In [22]:
#!pip install beautifulsoup4
from bs4 import BeautifulSoup

In [23]:
# Remove html tags
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

new_list = []
for rows in df['REVIEW_TEXT_str']:
    new_list.append(strip_html_tags(rows))
df['tag_removed'] = new_list

C:\Users\paolo\AppData\Local\Temp\ipykernel_13496\350002440.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [24]:
df.head()

,Label,Review_Text,no_contract,REVIEW_TEXT_str,tag_removed
0,__label1__,"useful When least you think so, this product w...","[useful, When, least, you, think, so,, this, p...","useful When least you think so, this product w...","useful When least you think so, this product w..."
1,__label1__,New era for batteries Lithium batteries are so...,"[New, era, for, batteries, Lithium, batteries,...",New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...
2,__label1__,doesn't swing very well. I purchased this swin...,"[does not, swing, very, well., I, purchased, t...",does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...
3,__label1__,Great computing! I was looking for an inexpens...,"[Great, computing!, I, was, looking, for, an, ...",Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...
4,__label1__,Only use twice a week I only use it twice a we...,"[Only, use, twice, a, week, I, only, use, it, ...",Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...


In [25]:
import re
# Removing Numbers
df['number_removed'] = df['tag_removed'].apply(lambda x:re.sub(r'\d+', '', x))

In [26]:
df.head()

,Label,Review_Text,no_contract,REVIEW_TEXT_str,tag_removed,number_removed
0,__label1__,"useful When least you think so, this product w...","[useful, When, least, you, think, so,, this, p...","useful When least you think so, this product w...","useful When least you think so, this product w...","useful When least you think so, this product w..."
1,__label1__,New era for batteries Lithium batteries are so...,"[New, era, for, batteries, Lithium, batteries,...",New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...
2,__label1__,doesn't swing very well. I purchased this swin...,"[does not, swing, very, well., I, purchased, t...",does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...
3,__label1__,Great computing! I was looking for an inexpens...,"[Great, computing!, I, was, looking, for, an, ...",Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...
4,__label1__,Only use twice a week I only use it twice a we...,"[Only, use, twice, a, week, I, only, use, it, ...",Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...


In [27]:
#!pip install nltk
import nltk
nltk.download('all')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\paolo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\paolo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\paolo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\paolo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\paolo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       tagge

In [28]:
# Tokenization
df['tokenized'] = df['number_removed'].apply(word_tokenize)

In [29]:
# Conversion to lowercase
df['lower'] = df['tokenized'].apply(lambda x: [word.lower() for word in x])

In [30]:
# Removing Punctuation
#import string
#punc = string.punctuation
#df['no_punc'] = df['lower'].apply(lambda x: [word for word in x if word not in punc])

In [31]:
#Removal of stop words
#from nltk.corpus import stopwords, wordnet
#stop_words = set(stopwords.words('english'))
#df['stopwords_removed'] = df['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])

In [32]:
# Parts of speech tagging

df['pos_tags'] = df['lower'].apply(nltk.tag.pos_tag)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

df['wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

In [33]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

In [34]:
df.head()

,Label,Review_Text,no_contract,REVIEW_TEXT_str,tag_removed,number_removed,tokenized,lower,pos_tags,wordnet_pos,lemmatized
0,__label1__,"useful When least you think so, this product w...","[useful, When, least, you, think, so,, this, p...","useful When least you think so, this product w...","useful When least you think so, this product w...","useful When least you think so, this product w...","[useful, When, least, you, think, so, ,, this,...","[useful, when, least, you, think, so, ,, this,...","[(useful, JJ), (when, WRB), (least, JJS), (you...","[(useful, a), (when, n), (least, a), (you, n),...","[useful, when, least, you, think, so, ,, this,..."
1,__label1__,New era for batteries Lithium batteries are so...,"[New, era, for, batteries, Lithium, batteries,...",New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...,"[New, era, for, batteries, Lithium, batteries,...","[new, era, for, batteries, lithium, batteries,...","[(new, JJ), (era, NN), (for, IN), (batteries, ...","[(new, a), (era, n), (for, n), (batteries, n),...","[new, era, for, battery, lithium, battery, be,..."
2,__label1__,doesn't swing very well. I purchased this swin...,"[does not, swing, very, well., I, purchased, t...",does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...,"[does, not, swing, very, well, ., I, purchased...","[does, not, swing, very, well, ., i, purchased...","[(does, VBZ), (not, RB), (swing, VB), (very, R...","[(does, v), (not, r), (swing, v), (very, r), (...","[do, not, swing, very, well, ., i, purchase, t..."
3,__label1__,Great computing! I was looking for an inexpens...,"[Great, computing!, I, was, looking, for, an, ...",Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...,"[Great, computing, !, I, was, looking, for, an...","[great, computing, !, i, was, looking, for, an...","[(great, JJ), (computing, VBG), (!, .), (i, NN...","[(great, a), (computing, v), (!, n), (i, n), (...","[great, compute, !, i, be, look, for, an, inex..."
4,__label1__,Only use twice a week I only use it twice a we...,"[Only, use, twice, a, week, I, only, use, it, ...",Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...,"[Only, use, twice, a, week, I, only, use, it, ...","[only, use, twice, a, week, i, only, use, it, ...","[(only, RB), (use, NN), (twice, RB), (a, DT), ...","[(only, r), (use, n), (twice, r), (a, n), (wee...","[only, use, twice, a, week, i, only, use, it, ..."


In [35]:
df.loc[df["Label"] == "__label1__", "Label"] = 0
df.loc[df["Label"] == "__label2__", "Label"] = 1

In [36]:
df.head()

,Label,Review_Text,no_contract,REVIEW_TEXT_str,tag_removed,number_removed,tokenized,lower,pos_tags,wordnet_pos,lemmatized
0,0,"useful When least you think so, this product w...","[useful, When, least, you, think, so,, this, p...","useful When least you think so, this product w...","useful When least you think so, this product w...","useful When least you think so, this product w...","[useful, When, least, you, think, so, ,, this,...","[useful, when, least, you, think, so, ,, this,...","[(useful, JJ), (when, WRB), (least, JJS), (you...","[(useful, a), (when, n), (least, a), (you, n),...","[useful, when, least, you, think, so, ,, this,..."
1,0,New era for batteries Lithium batteries are so...,"[New, era, for, batteries, Lithium, batteries,...",New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...,New era for batteries Lithium batteries are so...,"[New, era, for, batteries, Lithium, batteries,...","[new, era, for, batteries, lithium, batteries,...","[(new, JJ), (era, NN), (for, IN), (batteries, ...","[(new, a), (era, n), (for, n), (batteries, n),...","[new, era, for, battery, lithium, battery, be,..."
2,0,doesn't swing very well. I purchased this swin...,"[does not, swing, very, well., I, purchased, t...",does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...,does not swing very well. I purchased this swi...,"[does, not, swing, very, well, ., I, purchased...","[does, not, swing, very, well, ., i, purchased...","[(does, VBZ), (not, RB), (swing, VB), (very, R...","[(does, v), (not, r), (swing, v), (very, r), (...","[do, not, swing, very, well, ., i, purchase, t..."
3,0,Great computing! I was looking for an inexpens...,"[Great, computing!, I, was, looking, for, an, ...",Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...,Great computing! I was looking for an inexpens...,"[Great, computing, !, I, was, looking, for, an...","[great, computing, !, i, was, looking, for, an...","[(great, JJ), (computing, VBG), (!, .), (i, NN...","[(great, a), (computing, v), (!, n), (i, n), (...","[great, compute, !, i, be, look, for, an, inex..."
4,0,Only use twice a week I only use it twice a we...,"[Only, use, twice, a, week, I, only, use, it, ...",Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...,Only use twice a week I only use it twice a we...,"[Only, use, twice, a, week, I, only, use, it, ...","[only, use, twice, a, week, i, only, use, it, ...","[(only, RB), (use, NN), (twice, RB), (a, DT), ...","[(only, r), (use, n), (twice, r), (a, n), (wee...","[only, use, twice, a, week, i, only, use, it, ..."


In [ ]:
# Save the DataFrame to a JSON file
df.to_json('preprocessing_output.json', orient='records')